### **Actualización 1.6:**

---


*   Plot animado de varios cuerpos






# Import Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print('Enter the path:')
path = input()
import sys
sys.path.insert(0,'/content/drive/MyDrive/'+path)


# Upload modules manually

In [ ]:
from google.colab import files
files.upload()

# Import functions

In [ ]:
import numpy as np
import math
from numba import cuda
from scipy.optimize import newton
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

from Cauchy_Problem import Cauchy_problem
from Temporal_Schemes import Euler, Crank_Nicolson, RK4, Inverse_Euler, LeapFrog
from fNbody import F_NBody
from initialConditions import initial_cond
from Physics import Kepler

# Información de la GPU

In [ ]:
from numba import cuda
gpu = cuda.get_current_device()
print("name = %s" % gpu.name)
print("maxThreadsPerBlock = %s" % str(gpu.MAX_THREADS_PER_BLOCK))
print("maxBlockDimX = %s" % str(gpu.MAX_BLOCK_DIM_X))
print("maxBlockDimY = %s" % str(gpu.MAX_BLOCK_DIM_Y))
print("maxBlockDimZ = %s" % str(gpu.MAX_BLOCK_DIM_Z))
print("maxGridDimX = %s" % str(gpu.MAX_GRID_DIM_X))
print("maxGridDimY = %s" % str(gpu.MAX_GRID_DIM_Y))
print("maxGridDimZ = %s" % str(gpu.MAX_GRID_DIM_Z))
print("maxSharedMemoryPerBlock = %s" % str(gpu.MAX_SHARED_MEMORY_PER_BLOCK))
print("asyncEngineCount = %s" % str(gpu.ASYNC_ENGINE_COUNT))
print("canMapHostMemory = %s" % str(gpu.CAN_MAP_HOST_MEMORY))
print("multiProcessorCount = %s" % str(gpu.MULTIPROCESSOR_COUNT))
print("maxThreadsPerMultiprocessor = %s" % str(gpu.MAX_THREADS_PER_MULTI_PROCESSOR))
print("warpSize = %s" % str(gpu.WARP_SIZE))
print("unifiedAddressing = %s" % str(gpu.UNIFIED_ADDRESSING))
print("pciBusID = %s" % str(gpu.PCI_BUS_ID))
print("pciDeviceID = %s" % str(gpu.PCI_DEVICE_ID))

cuda.detect()

# main

In [ ]:
1print('Enter number of bodies:')
Nb = int(input()) # 1024*72*2 # nº cuerpos tiene que ser 1024 (nº threads por bloque) * 72 (nº bloques o SM) * A (nº entero a elegir)
Nc = 3 # nº coord

t0 = 0 # timepo inicial
print('Enter time [s]:')
tf = int(input()) # tiempo final
N = 10*tf # number of time steps
t = np.linspace(t0, tf, N)

U0, M = initial_cond(Nb,Nc)

U = Cauchy_problem(RK4, F_NBody, t, U0, Nb, M)

U_s = np.reshape( U, (N, Nb, 2, Nc) ) 
r = np.reshape( U_s[:, :, 0, :], (N, Nb, Nc) )


print('r',r)

print('Done!')





# 3D plot animation

In [ ]:
# 3D PLOT ANITAMED
def update_lines(num, walks, lines):
    for line, walk in zip(lines, walks):
        # NOTE: there is no .set_data() for 3 dim data...
        line.set_data(walk[:num, :2].T)
        line.set_3d_properties(walk[:num, 2])
    return lines

def paint(N,Nc,r,index):
  line=np.zeros((N,Nc))
  for i in range(N):
    line[i,:]=r[i,index,:]
  return line

# Data: Nb lines as (num_steps, 3) arrays
datas = [paint(N,Nc,r,index) for index in range(Nb)]
print ("datas: ",datas)

# Attaching 3D axis to the figure
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

# Create lines initially without data
lines = [ax.plot([], [], [])[0] for _ in datas]

# Setting the axes properties
ax.set(xlim3d=(-100, 100), xlabel='X')
ax.set(ylim3d=(-100, 100), ylabel='Y')
ax.set(zlim3d=(-100, 100), zlabel='Z')

# Creating the Animation object
line_ani = animation.FuncAnimation(fig, update_lines, N//2, fargs=(datas, lines), interval=2500)

if not path:
  f = r"/content/drive/MyDrive/animation1.mp4"
else:
  f = r"/content/drive/MyDrive/" + path + "/animation1.mp4" 
writervideo = animation.FFMpegWriter(fps=60) 
line_ani.save(f, writer=writervideo, dpi=300)

# Attaching 3D axis to the figure
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

# Create lines initially without data
lines = [ax.plot([], [], [])[0] for _ in datas]
dx = np.ceil(np.amin(np.amin(r[-1,:,0]))/10)*10
dy = np.ceil(np.amin(np.amin(r[-1,:,0]))/10)*10
dz = np.ceil(np.amin(np.amin(r[-1,:,0]))/10)*10
d = max(dx,dy,dz)
ax.set(xlim3d=(-1*d, 1*d), xlabel='X')
ax.set(ylim3d=(-1*d, 1*d), ylabel='Y')
ax.set(zlim3d=(-1*d, 1*d), zlabel='Z')

# Creating the Animation object
line_ani2 = animation.FuncAnimation(fig, update_lines, N//2, fargs=(datas, lines), interval=2500)

if not path:
  f2 = r"/content/drive/MyDrive/animation2.mp4"
else:
  f2 = r"/content/drive/MyDrive/" + path + "/animation2.mp4" 
writervideo = animation.FFMpegWriter(fps=60) 
line_ani2.save(f2, writer=writervideo, dpi=300)
#plt.show()

In [ ]:
# 3D PLOT FIJO
colors = ['b','r','g','m','y','c']
fig = plt.figure(2)
ax = fig.add_subplot(projection='3d')
ax.plot(r[:, 0, 0], r[:, 0, 1], r[:, 0, 2], colors[0])
ax.plot(r[:, 1, 0], r[:, 1, 1], r[:, 1, 2], colors[1])
ax.plot(r[:, 2, 0], r[:, 2, 1], r[:, 2, 2], colors[2])
ax.plot(r[:, 3, 0], r[:, 3, 1], r[:, 3, 2], colors[3])
plt.title(f'N = {Nb} body problem: 3D projection')
plt.xlabel("x")
plt.ylabel("y")

#plt.grid(True)
plt.show()